In [ ]:
%matplotlib inline


# Cox Regression


## Cox Proportional Hazrds Regression
Cox Proportional Hazrds (CoxPH) regression is to describe the survival according to several corvariates. The difference between CoxPH regression and Kaplan-Meier curves or the logrank tests is that the latter only focus on modeling the survival according to one factor (categorical predictor is best) while the former is able to take into consideration any covariates simultaneouly, regardless of whether they're quantitatrive or categorical. The model is as follow:

\begin{align}h(t) = h_0(t)\exp(\eta).\end{align}


where,

- $\eta = x\beta.$
- $t$ is the survival time.
- $h(t)$ is the hazard function which evaluate the risk of dying at time $t$.
- $h_0(t)$ is called the baseline hazard. It describes value of the hazard if all the predictors are zero.
- $beta$ measures the impact of covariates.


Consider two case $i$ and $i'$ that have different x values. Their hazard function can be simply written as follow

\begin{align}h_i(t) = h_0(t)\exp(\eta_i) = h_0(t)\exp(x_i\beta),\end{align}


and

\begin{align}h_{i'}(t) = h_0(t)\exp(\eta_{i'}) = h_0(t)\exp(x_{i'}\beta).\end{align}


The hazard ratio for these two cases is

\begin{align}\frac{h_i(t)}{h_{i'}(t)} & = \frac{h_0(t)\exp(\eta_i)}{h_0(t)\exp(\eta_{i'})} \\
                           & = \frac{\exp(\eta_i)}{\exp(\eta_{i'})},\end{align}



which is independent of time.

### Real Data Example (Lung Cancer Dataset)
We are going to apply best subset selection to the NCCTG Lung Cancer Dataset from [https://www.kaggle.com/ukveteran/ncctg-lung-cancer-data](https://www.kaggle.com/ukveteran/ncctg-lung-cancer-data). 
This dataset consists of survival informatoin of patients with advanced lung cancer from the North Central Cancer Treatment Group. The proportional hazards model allows the analysis of survival data by regression modeling. Linearity is assumed on the log scale of the hazard. The hazard ratio in Cox proportional hazard model is assumed constant. 

First, we load the data.



In [ ]:
import pandas as pd 

data = pd.read_csv('cancer.csv')
data = data.drop(data.columns[[0, 1]], axis = 1)
print(data.head())

Then we remove the rows containing any missing data. After that, we have a total of 168 observations.



In [ ]:
data = data.dropna()
print(data.shape)

Then we change the factors `ph.ecog` into dummy variables:



In [ ]:
data['ph.ecog'] = data['ph.ecog'].astype("category")
data = pd.get_dummies(data)
data = data.drop('ph.ecog_0.0', axis = 1)
print(data.head())

We split the dataset into a training set and a test set. The model is going to be built on the training set and later we will test the model performance on the test set.



In [ ]:
import numpy as np
np.random.seed(0)

ind = np.linspace(1, 168, 168) <= round(168*2/3)
train = np.array(data[ind])
test = np.array(data[~ind])

print('train size: ', train.shape[0])
print('test size:', test.shape[0])

#### Model Fitting
The `CoxPHSurvivalAnalysis()` function in the `abess` package allows you to perform best subset selection in a highly efficient way. 

By default, the function implements the abess algorithm with the support size (sparsity level) changing from 0 to $\min\{p,n/log(n)p \}$ and the best support size is determined by EBIC. You can change the tunging criterion by specifying the argument `ic_type` and the support size by `support_size`. The available tuning criterion now are gic, aic, bic, ebic. Here we give an example.



In [ ]:
from abess import CoxPHSurvivalAnalysis
model = CoxPHSurvivalAnalysis(ic_type = 'gic')
model.fit(train[:, 2:], train[:, :2])

After fitting, the coefficients are stored in `model.coef_`, and the non-zero values indicate the variables used in our model.



In [ ]:
print(model.coef_)

This result shows that 4 variables (the 2nd, 3rd, 7th, 8th, 9th) are chosen into the Cox model. Then a further analysis can be based on them. 



#### More on the results
Hold on, we aren’t finished yet. After get the estimator, we can further do more exploring work. For example, you can use some generic steps to quickly draw some information of those estimators.

Simply fix the `support_size` in different level, you can plot a path of coefficients like: 



In [ ]:
import matplotlib.pyplot as plt

coef = np.zeros((10, 9))
ic = np.zeros(10)
for s in range(10):
    model = CoxPHSurvivalAnalysis(support_size = s, ic_type = 'gic')
    model.fit(train[:, 2:], train[:, :2])
    coef[s, :] = model.coef_
    ic[s] = model.ic_

for i in range(9):
    plt.plot(coef[:, i], label = i)

plt.xlabel('support_size')
plt.ylabel('coefficients')
plt.legend()
plt.show()

Or a view of decreasing of information criterion:



In [ ]:
plt.plot(ic, 'o-')
plt.xlabel('support_size')
plt.ylabel('GIC')
plt.show()

Prediction is allowed for all the estimated model. Just call `predict()` function under the model you are interested in. The values it return are $\exp(\eta)=\exp(x\beta)$, which is part of Cox PH hazard function.

Here he give the prediction on the `test` data.



In [ ]:
pred = model.predict(test[:, 2:])
print(pred)

With these predictions, we can compute the hazard ratio between every two observations (by deviding their values). Or, we can also compute the C-Index for our model, i.e., the probability that, for a pair of randomly chosen comparable samples, the sample with the higher risk prediction will experience an event before the other sample or belong to a higher binary class. 



In [ ]:
from sksurv.metrics import concordance_index_censored
cindex = concordance_index_censored(test[:, 1] == 2, test[:, 0], pred)
print(cindex[0])

On this dataset, the C-index is about 0.68.



## R tutorial
For R tutorial, please view [https://abess-team.github.io/abess/articles/v05-coxreg.html](https://abess-team.github.io/abess/articles/v05-coxreg.html).

